In [ ]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path

def find_root():
    current = Path.cwd().resolve()
    for path in [current] + list(current.parents):
        if (path / "data").exists():
            return path
    return current

PROJECT_ROOT = find_root()
RAW_DIR = PROJECT_ROOT / "data" / "raw"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

print(f"Project Root: {PROJECT_ROOT}")

price_path = PROCESSED_DIR / "henryhub_weekly_returns.csv"
if not price_path.exists():
    raise FileNotFoundError("CRITICAL: Run Notebook 01 first!")

df_weekly = pd.read_csv(price_path)
df_weekly['date'] = pd.to_datetime(df_weekly['date'])

df_weekly['year'] = df_weekly['date'].dt.year
df_weekly['month'] = df_weekly['date'].dt.month

print(f"Loaded Prices: {df_weekly.shape}")

def process_dynamic_overview(filename):
   
    path = RAW_DIR / filename
    if not path.exists():
        print(f"Missing: {filename}")
        return pd.DataFrame(columns=['YYYYMM'])

    df = pd.read_csv(path)
    
    df['YYYYMM'] = pd.to_numeric(df['YYYYMM'], errors='coerce')
    df = df[df['YYYYMM'] % 100 <= 12] 
    df['YYYYMM'] = df['YYYYMM'].astype(int)
    
    
    df['Value'] = pd.to_numeric(df['Value'], errors='coerce')
    
    
    pivot_df = df.pivot(index='YYYYMM', columns='Description', values='Value').reset_index()
    
    print(f"   - Discovered {len(pivot_df.columns) - 1} unique variables in {filename}")
    return pivot_df

def process_weather(filename, col_name, desc_filter):
    path = RAW_DIR / filename
    if not path.exists(): return pd.DataFrame(columns=['YYYYMM', col_name])
    
    df = pd.read_csv(path)
    df = df[df["Description"] == desc_filter].copy()
    
    df['YYYYMM'] = pd.to_numeric(df['YYYYMM'], errors='coerce')
    df = df[df['YYYYMM'] % 100 <= 12]
    df['YYYYMM'] = df['YYYYMM'].astype(int)
    
    df[col_name] = pd.to_numeric(df['Value'], errors='coerce')
    return df[['YYYYMM', col_name]]

print("Processing Weather Data...")
df_hdd = process_weather("EIA_T01_11_HDD_by_CensusDivision.csv", "HDD", "Heating Degree-Days, United States")
df_cdd = process_weather("EIA_T01_12_CDD_by_CensusDivision.csv", "CDD", "Cooling Degree-Days, United States")

print("Processing Overview Data (Dynamic)...")
df_fundamentals = process_dynamic_overview("EIA_T04_01_Natural_Gas_Overview_Monthly.csv")

print("Merging all datasets...")
master = df_weekly.merge(df_hdd, on='YYYYMM', how='left')
master = master.merge(df_cdd, on='YYYYMM', how='left')
master = master.merge(df_fundamentals, on='YYYYMM', how='left')


master = master.drop(columns=['YYYYMM'])

save_path = PROCESSED_DIR / "FINAL_weekly_natural_gas_data.csv"
master.to_csv(save_path, index=False)

print(f"Success! Saved {len(master)} rows to: {save_path}")
print(f"Total Columns: {len(master.columns)}")

required_cols = ['HDD', 'price', 'Natural Gas Exports', 'Natural Gas Production (Dry)']
missing = [c for c in required_cols if c not in master.columns]

if missing:
    print(f"WARNING: The following columns expected by Analysis are missing: {missing}")
else:
    print("All columns required for Analysis (03) are present.")

display(master.head())

Project Root: C:\Users\gefer\OneDrive - Univerzita Karlova\Natural Gas python\Final Project\Python-Final-Project-Natural-gas-price-analysis-
Loaded Prices: (1513, 6)
Processing Weather Data...
Processing Overview Data (Dynamic)...
   - Discovered 11 unique variables in EIA_T04_01_Natural_Gas_Overview_Monthly.csv
Merging all datasets...
Success! Saved 1513 rows to: C:\Users\gefer\OneDrive - Univerzita Karlova\Natural Gas python\Final Project\Python-Final-Project-Natural-gas-price-analysis-\data\processed\FINAL_weekly_natural_gas_data.csv
Total Columns: 18
All columns required for Analysis (03) are present.


,date,price,log_return,year,month,HDD,CDD,NGPL Production,Natural Gas Balancing Item,Natural Gas Consumption,Natural Gas Exports,Natural Gas Gross Withdrawals,Natural Gas Imports,Natural Gas Marketed Production (Wet),Natural Gas Net Imports,Natural Gas Net Storage Withdrawals,Natural Gas Production (Dry),Supplemental Gaseous Fuels
0,1997-01-17,3.91,-0.002554,1997,1,935.0,8.0,82.811,-69.813,2540.341,12.028,2086.781,278.288,1707.002,266.259,709.028,1624.195,10.672
1,1997-01-24,2.62,-0.400363,1997,1,935.0,8.0,82.811,-69.813,2540.341,12.028,2086.781,278.288,1707.002,266.259,709.028,1624.195,10.672
2,1997-01-31,2.77,0.055673,1997,1,935.0,8.0,82.811,-69.813,2540.341,12.028,2086.781,278.288,1707.002,266.259,709.028,1624.195,10.672
3,1997-02-07,2.39,-0.147554,1997,2,677.0,12.0,75.031,193.108,2273.601,12.443,1903.271,240.545,1546.616,228.102,371.446,1471.590,9.354
4,1997-02-14,2.12,-0.119877,1997,2,677.0,12.0,75.031,193.108,2273.601,12.443,1903.271,240.545,1546.616,228.102,371.446,1471.590,9.354
